Development of the data pipeline for the raw measurements from Project Breathe

In [4]:
import pandas as pd
import src.data.helpers as dh
import src.data.breathe_data as breathe_data

In [5]:
path = (
    dh.get_path_to_main()
    + "DataFiles/BR/MeasurementData/Breathe_Spirometer_20231113.csv"
)
df = pd.read_csv(path)

In [6]:
print(df.columns)
print(df.shape)

Index(['UserId', 'EntityId', 'Timestamp', 'ClientTimestamp', 'IsDeleted',
       'CaptureType', 'CaptureDeviceMake', 'CaptureDeviceModel',
       'ThirdPartyId', 'PEF', 'PEFTime', 'FEV6', 'FEV1', 'FEV1PersonalBest',
       'FEV075', 'FEV1DivFEV6', 'FEV1DivFVC', 'FEF2575', 'FEV1Percent', 'EVol',
       'FVC', 'InvalidEntry', 'VolumeTimeCurve', 'FlowVolumeCurve',
       'IsCapturePrimary', 'CaptureSessionId'],
      dtype='object')
(104324, 26)


In [7]:
df.IsDeleted.unique()

array([False,  True])

In [20]:
tmp_size = df.shape[0]
df0 = df[~df.IsDeleted]
print(f"Removed {tmp_size - df0.shape[0]} entries manually deleted on the app")

Removed 0 entries manually deleted on the app


In [21]:
df0.describe()

,EntityId,PEF,PEFTime,FEV6,FEV1,FEV1PersonalBest,FEV075,FEV1DivFEV6,FEV1DivFVC,FEF2575,FEV1Percent,EVol,FVC
count,1.017590e+05,87122.000000,4549.0,100680.000000,101759.000000,96319.000000,96319.000000,96319.000000,4163.000000,98277.000000,101176.000000,4549.0,7634.000000
mean,1.023662e+06,279.116710,0.0,3.029517,2.130394,3.262218,1.896338,0.681364,30.227024,1.631205,60.820797,0.0,1.116860
std,3.904265e+04,199.826929,0.0,1.068609,0.880368,0.979416,0.857896,0.176934,33.537712,1.169019,25.287741,0.0,1.790167
min,1.000001e+06,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,1.002048e+06,147.000000,0.0,2.340000,1.560000,2.900000,1.420000,0.620000,0.000000,0.900000,45.000000,0.0,0.000000
50%,1.008223e+06,264.000000,0.0,2.770000,1.910000,3.500000,1.710000,0.690000,0.000000,1.190000,59.000000,0.0,0.000000
75%,1.029546e+06,435.000000,0.0,3.650000,2.620000,3.900000,2.350000,0.800000,61.700001,2.130000,77.000000,0.0,1.980000
max,1.336051e+06,966.000000,0.0,9.830000,6.030000,5.300000,5.780000,1.000000,100.000000,8.680000,700.000000,0.0,7.130000


In [22]:
def count_na(series):
    return series.isna().sum()


df0.agg(count_na)

UserId                     0
EntityId                   0
Timestamp                  0
ClientTimestamp            0
IsDeleted                  0
CaptureType                0
CaptureDeviceMake       5946
CaptureDeviceModel      5977
ThirdPartyId          101151
PEF                    14637
PEFTime                97210
FEV6                    1079
FEV1                       0
FEV1PersonalBest        5440
FEV075                  5440
FEV1DivFEV6             5440
FEV1DivFVC             97596
FEF2575                 3482
FEV1Percent              583
EVol                   97210
FVC                    94125
InvalidEntry               0
VolumeTimeCurve        99802
FlowVolumeCurve        99801
IsCapturePrimary           0
CaptureSessionId           0
Datetime recorded          0
dtype: int64

In [23]:
df0["Datetime recorded"] = pd.to_datetime(df0.ClientTimestamp, utc=False)

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_29889/2387945828.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df0["Datetime recorded"] = pd.to_datetime(df0.ClientTimestamp, utc=False)


In [24]:
cols_to_keep = [
    "UserId",
    "Datetime recorded",
    "FEV1",
    "FEF2575",
    "PEF",
    "InvalidEntry",
]
df1 = df0[cols_to_keep]

In [25]:
df1 = df1[df1.InvalidEntry == False]

In [26]:
df1.head()

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry
0,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-07-29 16:19:05.125475+00:00,2.73,2.53,326.0,False
1,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 09:44:37.358766+00:00,2.68,2.49,353.0,False
2,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 09:45:39.029077+00:00,2.70,2.34,351.0,False
3,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 09:45:59.073886+00:00,2.88,NaN,NaN,False
4,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 16:25:24.516605+00:00,2.57,2.52,333.0,False


In [36]:
# Plot PEF histogram with plotly
import plotly.express as px

# Remove where PEF is 0
dftmp = df1[df1.PEF != 0]

fig = px.histogram(dftmp, x="PEF", nbins=500)
fig.update_layout(
    title="Histogram of PEF recordings from 25.05.2021 to 13.11.2023 (removed isDeleted and invalidEntry entries)"
)
fig.show()

In [38]:
dftmp[dftmp.PEF < 30]

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry
9769,70bae720-0865-4ab8-99ef-2d569a036963,2022-08-21 12:21:16.244508+00:00,0.43,0.40,26.0,False
9784,70bae720-0865-4ab8-99ef-2d569a036963,2022-10-02 09:38:28.991220+00:00,0.42,0.39,26.0,False
9803,70bae720-0865-4ab8-99ef-2d569a036963,2023-02-07 10:24:26.764135+00:00,0.42,0.36,26.0,False
9804,70bae720-0865-4ab8-99ef-2d569a036963,2023-02-07 10:24:26.764185+00:00,0.40,0.34,25.0,False
9873,70bae720-0865-4ab8-99ef-2d569a036963,2023-04-03 08:34:32.169147+01:00,0.41,0.41,25.0,False
9881,70bae720-0865-4ab8-99ef-2d569a036963,2023-04-09 08:24:31.187834+01:00,0.40,0.32,25.0,False
9891,70bae720-0865-4ab8-99ef-2d569a036963,2023-04-23 09:21:17.086919+01:00,0.41,0.37,25.0,False
13076,511950cb-3b80-4b67-9e19-35bad5649798,2023-09-07 20:36:30.004334+01:00,0.37,0.35,19.0,False
13334,511950cb-3b80-4b67-9e19-35bad5649798,2023-11-07 21:41:49.905547+00:00,0.29,0.33,19.0,False
25853,7eb62e50-e389-4c4f-8cf8-5574dc68c3f6,2022-11-07 14:27:38.278619+00:00,0.27,0.66,18.0,False


In [13]:
def get_BR_ID_to_partition_key_mapping():
    df = pd.read_excel(
        dh.get_path_to_main() + "ExcelFiles/BR/PredModInputData.xlsx",
        sheet_name="brPatient",
        usecols="A, AD",
    )
    # Set ID as string
    df.ID = df.ID.astype(str)
    return df


df_key_map = get_BR_ID_to_partition_key_mapping()

In [14]:
# Is mapping complete?
keys_intersect = set(df1.UserId).intersection(set(df_key_map.PartitionKey))
print(f"{len(keys_intersect)} intersecting keys")
keys_missing_in_map = set(df1.UserId) - set(df_key_map.PartitionKey)
print(f"{len(keys_missing_in_map)} keys from physdata are missing in mapping table")
keys_missing_in_physdata = set(df_key_map.PartitionKey) - set(df1.UserId)
print(
    f"{len(keys_missing_in_physdata)} keys from mapping table are missing in phys data"
)

239 intersecting keys
215 keys from physdata are missing in mapping table
18 keys from mapping table are missing in phys data


In [15]:
df2 = df1.merge(df_key_map, left_on="UserId", right_on="PartitionKey", how="inner")
df2.ID.nunique()

240

In [16]:
df2

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID,PartitionKey
0,18b5c897-2445-4390-bfbe-019b70c20fda,2020-02-03 17:14:00.583188+00:00,2.44,1.18,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
1,18b5c897-2445-4390-bfbe-019b70c20fda,2020-02-05 18:09:50.357438+00:00,2.52,1.35,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
2,18b5c897-2445-4390-bfbe-019b70c20fda,2020-02-11 20:13:46.856589+00:00,2.52,1.35,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
3,18b5c897-2445-4390-bfbe-019b70c20fda,2020-02-20 17:45:24.565244+00:00,2.84,0.00,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
4,18b5c897-2445-4390-bfbe-019b70c20fda,2020-02-21 18:38:11.955885+00:00,2.82,1.53,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
...,...,...,...,...,...,...,...,...
76714,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 06:30:15.415189+00:00,3.41,3.04,483.0,True,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76715,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 06:30:15.415209+00:00,3.27,2.97,492.0,False,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76716,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 06:31:57.182137+00:00,3.29,3.15,471.0,True,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76717,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 06:32:50.482851+00:00,3.31,3.09,472.0,False,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316


In [17]:
df1.shape

(101759, 6)

In [18]:
# Why 1 off error??

In [19]:
df3 = df2.drop(columns=["UserId", "PartitionKey"])

In [20]:
df3

,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID
0,2020-02-03 17:14:00.583188+00:00,2.44,1.18,0.0,False,183
1,2020-02-05 18:09:50.357438+00:00,2.52,1.35,0.0,False,183
2,2020-02-11 20:13:46.856589+00:00,2.52,1.35,0.0,False,183
3,2020-02-20 17:45:24.565244+00:00,2.84,0.00,0.0,False,183
4,2020-02-21 18:38:11.955885+00:00,2.82,1.53,0.0,False,183
...,...,...,...,...,...,...
76714,2022-08-10 06:30:15.415189+00:00,3.41,3.04,483.0,True,273
76715,2022-08-10 06:30:15.415209+00:00,3.27,2.97,492.0,False,273
76716,2022-08-10 06:31:57.182137+00:00,3.29,3.15,471.0,True,273
76717,2022-08-10 06:32:50.482851+00:00,3.31,3.09,472.0,False,273


In [23]:
df3["PEF"] = df2.PEF / 60

In [22]:
# Understand invalid entry

df3.ID.value_counts()
df3["Date recorded"] = df3["Datetime recorded"]
df_for_ID = df3[df3.ID == "101"].reset_index()
df_for_ID.InvalidEntry.value_counts()

InvalidEntry
True     12275
False     3900
Name: count, dtype: int64

In [ ]:
# Has invalid entry
df_for_ID[df_for_ID["Date recorded"] == df_for_ID["Date recorded"][0]]

,index,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID,Date recorded
0,49269,2022-08-04 17:16:15.691332000,1.31,0.54,0.0,False,101,2022-08-04
1,49270,2022-08-04 17:16:15.692332100,1.31,0.57,0.0,False,101,2022-08-04
2,49271,2022-08-04 17:16:15.692332100,1.31,0.67,0.0,False,101,2022-08-04
3,49272,2022-08-04 17:16:15.692332100,1.30,0.69,0.0,False,101,2022-08-04
4,49273,2022-08-04 17:16:15.692332100,1.28,0.60,0.0,False,101,2022-08-04
...,...,...,...,...,...,...,...,...
5720,54989,2022-08-04 08:32:02.173501500,1.64,0.88,176.0,True,101,2022-08-04
5721,54990,2022-08-04 08:32:02.173501500,1.54,0.91,168.0,False,101,2022-08-04
5722,54991,2022-08-04 08:32:02.173501500,1.67,0.97,172.0,True,101,2022-08-04
5723,54992,2022-08-04 08:32:02.173501500,1.63,0.94,170.0,True,101,2022-08-04


# Data post processing

In [2]:
df = breathe_data.build_O2_FEV1_df()


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  48976
Number of FEV1 recordings: 41791
Number of O2 Saturation recordings: 43930
Dropped 12231 entries with at least one NaN in subset ['O2 Saturation', 'FEV1']
This includes dropping 5046 entries with NaN O2 Saturation
This includes dropping 7185 entries with NaN FEV1
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted shou

In [3]:
df_old = breathe_data.build_O2_FEV1_df(meas_file=1)


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  233
Number of rows:  26812
Number of FEV1 recordings: 23778
Number of O2 Saturation recordings: 23431
Dropped 6415 entries with at least one NaN in subset ['O2 Saturation', 'FEV1']
This includes dropping 3381 entries with NaN O2 Saturation
This includes dropping 3034 entries with NaN FEV1
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted shoul

In [3]:
# df_old.to_excel(dh.get_path_to_main() + "ExcelFiles/BR/BR_O2_FEV1_FEF2575.xlsx", index=False)

# Processing with PEF

In [2]:
df = breathe_data.build_O2_FEV1_FEF2575_df()


*** Building O2Sat, FEV1, FEF2575 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  48978
Number of FEV1 recordings: 41791
Number of FEF2575 recordings: 37068
Number of PEF recordings: 16755
Number of O2 Saturation recordings: 43930
Dropped 33610 entries with at least one NaN in subset ['O2 Saturation', 'FEV1', 'FEF2575', 'PEF']
This includes dropping 5048 entries with NaN O2 Saturation
This includes dropping 7187 entries with NaN FEV1
This includes dropping 11910 entries with

In [7]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),FEF2575%PEF
0,101,2021-05-25,1.68,98.0,1.17,227.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.536607,100.874827,3.783333,30.925110
1,101,2021-05-26,1.65,98.0,1.06,236.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,45.705597,100.874827,3.933333,26.949153
2,101,2021-05-27,1.69,98.0,1.12,183.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.813611,100.874827,3.050000,36.721311
3,101,2021-05-28,1.67,98.0,1.08,175.0,1.69,53,Male,173.0,3.610061,97.150104,46.813611,46.259604,100.874827,2.916667,37.028571
4,101,2021-05-29,1.69,98.0,1.16,171.0,1.76,53,Male,173.0,3.610061,97.150104,48.752636,46.813611,100.874827,2.850000,40.701754


In [6]:
df["PEF (L/s)"] = df.PEF / 60
df["FEF2575%PEF"] = df["FEF2575"] / df["PEF (L/s)"] * 100

In [9]:
df.describe()

,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),FEF2575%PEF
count,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000,15368.000000
mean,2.297343,97.283837,1.910049,409.511127,2.357864,39.432197,165.873731,3.383367,97.728202,69.516518,67.690617,99.549021,6.825185,27.277097
std,0.857375,1.741286,1.213329,137.509160,0.861811,11.252491,8.758594,0.657402,0.520266,19.959031,19.924648,1.903121,2.291819,12.487859
min,0.590000,75.000000,0.350000,62.000000,0.850000,18.000000,143.000000,2.213968,96.791365,24.551342,21.316188,76.301768,1.033333,4.228433
25%,1.690000,97.000000,1.040000,326.000000,1.730000,31.000000,159.000000,2.887169,97.177010,52.853943,50.656898,98.834405,5.433333,17.078512
50%,2.090000,98.000000,1.550000,406.000000,2.150000,38.000000,165.000000,3.121961,98.042804,69.541521,67.719804,99.864990,6.766667,25.099602
75%,2.810000,98.000000,2.490000,496.000000,2.890000,52.000000,173.000000,3.883230,98.204237,82.373860,80.021186,100.810314,8.266667,35.122785
max,5.300000,100.000000,6.920000,966.000000,5.300000,64.000000,193.000000,5.322753,98.509166,144.814571,144.814571,103.104825,16.100000,207.123288


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,PEF (L/s),FEF2575%PEF
8234,220,2023-05-05,1.74,100.0,6.92,406.0,1.79,41,Female,166.0,3.128822,98.096615,57.210025,55.611979,101.940317,6.766667,102.266010
10145,242,2021-06-28,3.17,98.0,3.87,213.0,3.17,39,Female,165.0,3.138644,98.114552,100.999034,100.999034,99.883247,3.550000,109.014085
10300,242,2022-06-20,2.66,99.0,2.52,73.0,2.66,39,Female,165.0,3.138644,98.114552,84.749978,84.749978,100.902463,1.216667,207.123288
10688,250,2022-07-21,4.57,98.0,5.28,260.0,4.57,37,Male,169.0,3.849069,97.221852,118.730020,118.730020,100.800384,4.333333,121.846154
10768,250,2023-02-01,4.44,98.0,4.75,291.0,4.44,37,Male,169.0,3.849069,97.221852,115.352580,115.352580,100.800384,4.850000,97.938144
10781,250,2023-03-08,4.21,97.0,4.75,201.0,4.26,37,Male,169.0,3.849069,97.221852,110.676124,109.377109,99.771808,3.350000,141.791045


In [19]:
df.to_excel(
    f"{dh.get_path_to_main()}ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF.xlsx", index=False
)